In [ ]:
#| default_exp models.rnn
#| default_cls_lvl 3

In [ ]:
#| export
from tsfast.data import *
from tsfast.models.layers import *
from tsfast.learner.callbacks import *
from tsfast.learner.losses import *

from fastai.basics import *

In [ ]:
from nbdev.config import get_config

In [ ]:
project_root = get_config().config_file.parent
f_path = project_root / 'test_data/WienerHammerstein'

In [ ]:
seq = DataBlock(blocks=(SequenceBlock.from_hdf(['u','y'],TensorSequencesInput,clm_shift=[0,-1]),
                        SequenceBlock.from_hdf(['y'],TensorSequencesOutput,clm_shift=[-1])),
                 get_items=CreateDict([DfHDFCreateWindows(win_sz=100+1,stp_sz=100,clm='u')]),
                 splitter=ApplyToDict(ParentSplitter()))
db = seq.dataloaders(get_hdf_files(f_path))

## RNNs

### Basic RNN

In [ ]:
#| export
from fastai.text.models.awdlstm import RNNDropout,WeightDropout

class RNN(nn.Module):
    "inspired by https://arxiv.org/abs/1708.02182"

    def __init__(self, input_size,hidden_size, num_layers, 
                 hidden_p=0.0, input_p=0.0, weight_p=0.0,
                 rnn_type='gru',ret_full_hidden=False,stateful=False,
                 normalization='',**kwargs):
        super().__init__()
        store_attr()
        self.bs = 1
        
        self.rnns = nn.ModuleList([self._one_rnn(input_size if l == 0 else hidden_size,
                                                 hidden_size,weight_p,rnn_type,**kwargs) for l in range(num_layers)])
        
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(num_layers)])
        
        if normalization == '':
            self.norm_layers = [None]*num_layers
        elif normalization == 'layernorm':
            self.norm_layers = nn.ModuleList([nn.LayerNorm(hidden_size,elementwise_affine=False) for l in range(num_layers)])
        elif normalization == 'batchnorm':
            self.norm_layers = nn.ModuleList([(BatchNorm_1D_Stateful(hidden_size,stateful=stateful,batch_first=True,affine=False)) for i in range(num_layers)])  
        else:
            raise ValueError('Invalid value for normalization')
        self.reset_state()

    def forward(self, inp, h_init=None):
        bs,seq_len,_ = inp.shape
        if h_init is None and self.stateful: h_init = self._get_hidden(bs)
                
        r_input = self.input_dp(inp) if self.input_p > 0 else inp
        full_hid,new_hidden = [],[]
#         import pdb; pdb.set_trace()
        for l, (rnn,hid_dp,nrm) in enumerate(zip(self.rnns,self.hidden_dps,self.norm_layers)):
            r_output, h = rnn(r_input.contiguous(),h_init[l] if h_init is not None else None)
            
            if self.normalization != '':
                r_output = nrm(r_output)
                
            if l != self.num_layers - 1: 
                r_output = hid_dp(r_output)
            
            full_hid.append(r_output)
            new_hidden.append(h)
            r_input = r_output

        if self.stateful:
            self.hidden =  to_detach(new_hidden, cpu=False, gather=False)
            self.bs = bs
        output = r_output if not self.ret_full_hidden else torch.stack(full_hid, 0)
        
        return output, new_hidden

    def _get_hidden(self,bs):
        '''retrieve internal hidden state, check if model device has changed'''
        if self.hidden is None: return None
        if bs!=self.bs: return None
        if self.hidden[0][0].device != one_param(self).device: return None
#         import pdb; pdb.set_trace()
        return self.hidden
    
    def _one_rnn(self, n_in, n_out, weight_p, rnn_type,**kwargs):
        "Return one of the inner rnn"
        if rnn_type == 'gru':
            rnn = nn.GRU(n_in, n_out,1,batch_first=True,**kwargs)
            if weight_p > 0: 
                rnn = WeightDropout(rnn,weight_p)
        elif rnn_type == 'lstm':
            rnn = nn.LSTM(n_in, n_out,1,batch_first=True,**kwargs)
            if weight_p > 0: 
                rnn = WeightDropout(rnn,weight_p)
        elif rnn_type == 'rnn':
            rnn = nn.RNN(n_in, n_out,1,batch_first=True,**kwargs)
            if weight_p > 0: 
                rnn = WeightDropout(rnn,weight_p)
        else:
            raise Exception
        return rnn
    
    def reset_state(self):
        self.hidden = None

In [ ]:
#| export
class Sequential_RNN(RNN):
    '''RNN Variant for Sequential Modules'''
    def forward(self, inp, h_init=None):
        return super().forward(inp, h_init)[0]

In [ ]:
#| export
class SimpleRNN(nn.Module):
    
    @delegates(RNN, keep=True)
    def __init__(self,input_size,output_size,num_layers=1,hidden_size=100,linear_layers=0,return_state=False,**kwargs):
        super().__init__()
        self.return_state = return_state
        self.rnn = RNN(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers,**kwargs)
        self.final = SeqLinear(hidden_size,output_size,hidden_size=hidden_size,hidden_layer=linear_layers,act=nn.LeakyReLU)
    def forward(self, x, h_init=None):
        out,h = self.rnn(x,h_init)
        out = self.final(out)
        return out if not self.return_state else (out,h)

In [ ]:
model = SimpleRNN(2,1,2,stateful=False,normalization='batchnorm')
lrn = Learner(db,model,loss_func=nn.MSELoss())#.fit(10)

In [ ]:
model = SimpleRNN(2,1,2,rnn_type='lstm')
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,0.032483,0.012665,00:00


In [ ]:
model = SimpleRNN(2,1,2,rnn_type='gru')
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(2)

epoch,train_loss,valid_loss,time
0,0.018673,0.005693,00:02
1,0.010276,0.002584,00:02


### RNN Learner
The Learner includes model specific optimizations. Removing the first n_skip samples of the loss function of transient time, greatly improves training stability.


In [ ]:
#| export
from tsfast.data.loader import get_inp_out_size


In [ ]:
#| export
@delegates(SimpleRNN, keep=True)
def RNNLearner(dls,loss_func=nn.L1Loss(),metrics=[fun_rmse],n_skip=0,num_layers=1,hidden_size=100,stateful=False,opt_func=Adam,cbs=None,**kwargs):
    if cbs is None: cbs = []

    inp,out = get_inp_out_size(dls)
    model = SimpleRNN(inp,out,num_layers,hidden_size,stateful=stateful,**kwargs)
  
    skip = partial(SkipNLoss,n_skip=n_skip)
        
    metrics= [skip(f) for f in metrics]

    if stateful: 
        cbs.append(TbpttResetCB())
        # if stateful apply n_skip with a callback for the first minibatch of a tbptt sequence
        cbs.append(SkipFirstNCallback(n_skip))
    else:
        loss_func = skip(loss_func)
        
    lrn = Learner(dls,model,loss_func=loss_func,opt_func=opt_func,metrics=metrics,cbs=cbs,lr=3e-3)
    return lrn


In [ ]:
from tsfast.datasets.core import create_dls_test
dls = create_dls_test(prediction=True).cpu()
RNNLearner(dls,rnn_type='gru').fit(1,1e-4)


epoch,train_loss,valid_loss,fun_rmse,time
0,0.185556,0.186565,0.231737,00:00


### Autoregressive RNN Learner


In [ ]:
#| export
@delegates(SimpleRNN, keep=True)
def AR_RNNLearner(dls,alpha=0,beta=0,early_stop=0,metrics=None,n_skip=0,opt_func=Adam,**kwargs):
    skip = partial(SkipNLoss,n_skip=n_skip)
    
    inp,out = get_inp_out_size(dls)
    model = AR_Model(SimpleRNN(inp+out,out,**kwargs),ar=False)
    model.init_normalize(dls.one_batch())
    
    cbs=[ARInitCB(),TimeSeriesRegularizer(alpha=alpha,beta=beta,modules=[model.model.rnn])]#SaveModelCallback()
    if early_stop > 0:
        cbs += [EarlyStoppingCallback(patience=early_stop)]
        
    if metrics is None: metrics=SkipNLoss(fun_rmse,n_skip)
        
    lrn = Learner(dls,model,loss_func=nn.L1Loss(),opt_func=opt_func,metrics=metrics,cbs=cbs,lr=3e-3)
    return lrn


In [ ]:
AR_RNNLearner(dls).fit(1)


epoch,train_loss,valid_loss,fun_rmse,time
0,0.121968,0.071209,0.089933,00:00


### Residual RNN

In [ ]:
#| export
class ResidualBlock_RNN(nn.Module):
    
    @delegates(RNN, keep=True)
    def __init__(self, input_size,hidden_size,**kwargs):
        super().__init__()
        self.rnn1 = RNN(input_size,hidden_size,num_layers=1,**kwargs)
        self.rnn2 = RNN(hidden_size,hidden_size,num_layers=1,**kwargs)
        self.residual = SeqLinear(input_size,hidden_size,hidden_layer=0) if hidden_size!=input_size else noop

    def forward(self, x):
        out,_ = self.rnn1(x)
        out,_ = self.rnn2(out)
        return out+self.residual(x)

In [ ]:
#| export
class SimpleResidualRNN(nn.Sequential):
    
    @delegates(ResidualBlock_RNN, keep=True)
    def __init__(self, input_size,output_size,num_blocks=1,hidden_size=100,**kwargs):
        super().__init__()
        for i in range(num_blocks):
            self.add_module('rnn%d'%i, ResidualBlock_RNN(input_size if i==0 else hidden_size,hidden_size,**kwargs))

        self.add_module('linear', SeqLinear(hidden_size,output_size,hidden_size,hidden_layer=1))

In [ ]:
model = SimpleResidualRNN(2,1,1,stateful=False,normalization='')
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,0.020712,0.000780,00:02


### Dense RNN

In [ ]:
#| export
class DenseLayer_RNN(nn.Module):
    
    @delegates(RNN, keep=True)
    def __init__(self, input_size,hidden_size,**kwargs):
        super().__init__()
        self.rnn1 = RNN(input_size,hidden_size,num_layers=1,**kwargs)
        self.rnn2 = RNN(hidden_size,hidden_size,num_layers=1,**kwargs)

    def forward(self, x):
        out,_ = self.rnn1(x)
        out,_ = self.rnn2(out)
        return torch.cat([x, out], 2) 
    
class DenseBlock_RNN(nn.Sequential):
    @delegates(DenseLayer_RNN, keep=True)
    def __init__(self, num_layers, num_input_features,growth_rate,**kwargs):
        super().__init__()
        for i in range(num_layers):
            self.add_module('denselayer%d' % i, DenseLayer_RNN(num_input_features + i * growth_rate,growth_rate,**kwargs))
            

In [ ]:
#| export
class DenseNet_RNN(nn.Sequential):
    
    @delegates(RNN, keep=True)
    def __init__(self, input_size, output_size, growth_rate=32, block_config=(3,3),
                 num_init_features=32,**kwargs):

        super().__init__()
        self.add_module('rnn0', Sequential_RNN(input_size, num_init_features,1,**kwargs))      

        # Each denseblock
        num_features = num_init_features  
        for i, num_layers in enumerate(block_config):
            self.add_module('denseblock%d' % i, DenseBlock_RNN(num_layers, num_features,growth_rate=growth_rate,**kwargs))
            num_features=num_features + num_layers * growth_rate
            
            self.add_module('transition%d' % i,  Sequential_RNN(num_features, num_features // 2,1,**kwargs))
            num_features = num_features // 2
        self.add_module('final',  SeqLinear(num_features, output_size ,hidden_layer=0))

In [ ]:
model = DenseNet_RNN(2,1,growth_rate=10,block_config=(1,1),num_init_features=2)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,0.051308,0.047723,00:07


### Seperate RNN

In [ ]:
#| export
class SeperateRNN(nn.Module):
    
    @delegates(RNN, keep=True)
    def __init__(self,input_list,output_size,num_layers=1,hidden_size=100,linear_layers=1,**kwargs):
        super().__init__()
        self.input_list = input_list
        
        rnn_width = hidden_size//len(input_list)
        
        self.rnns =nn.ModuleList([ RNN(input_size=len(n),hidden_size=rnn_width,num_layers=1,**kwargs)
                        for n in input_list])
        
        self.rnn =RNN(input_size=rnn_width*len(input_list),hidden_size=hidden_size,
                      num_layers=num_layers,**kwargs)
        self.final = SeqLinear(hidden_size,output_size,hidden_size=hidden_size,hidden_layer=linear_layers)

    def forward(self, x):      
        rnn_out = [rnn(x[...,group])[0] for rnn,group in zip(self.rnns,self.input_list)]
        out = torch.cat(rnn_out,dim=-1)
        out,_ = self.rnn(out)
        out = self.final(out)
        return out

In [ ]:
#| include: false
import nbdev; nbdev.nbdev_export()